<a href="https://colab.research.google.com/github/firesh/micrograd_exercises/blob/main/micrograd_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# micrograd exercises

1. watch the [micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. come back and complete these exercises to level up :)

In [2]:
!pip install torch

## section 1: derivatives

In [2]:
# here is a mathematical expression that takes 3 inputs and produces one output
from math import sin, cos

def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5

print(f(2, 3, 4))

6.336362190988558


In [3]:
# write the function df that returns the analytical gradient of f
# i.e. use your skills from calculus to take the derivative, then implement the formula
# if you do not calculus then feel free to ask wolframalpha, e.g.:
# https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

def gradf(a, b, c):
  dfda = -3*a**2-0.5*a**-0.5
  dfdb = 2.5*b**1.5 + 3*cos(3*b)
  dfdc = c**-2
  return [dfda, dfdb, dfdc] # todo, return [df/da, df/db, df/dc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [4]:
# 1st try with micrograd

# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell
import math

from micrograd.engine import Value


def _sin(self):
    out = Value(math.sin(self.data), (self,), 'sin')

    def _backward():
      # print('sin', self.data, math.cos(self.data))
      self.grad += math.cos(self.data)

    out._backward = _backward
    return out

Value.sin = _sin


a = Value(2)
b = Value(3)
c = Value(4)

# -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5
o = -a**3 + (3*b).sin() - 1.0/c + b**2.5 - a**0.5

o.backward()
print(a, b, c)
# -----------
numerical_grad = [a.grad, b.grad, c.grad] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


Value(data=2, grad=-12.353553390593273) Value(data=3, grad=10.25699027111255) Value(data=4, grad=0.0625)
OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


In [21]:
# 2nd try with approximation

# now estimate the gradient numerically without any calculus, using
# the approximation we used in the video.
# you should not call the function df from the last cell

h = 0.000001
dfda = (f(2 + h, 3, 4) - f(2, 3, 4)) / h
dfdb = (f(2, 3 + h, 4) - f(2, 3, 4)) / h
dfdc = (f(2, 3, 4 + h) - f(2, 3, 4)) / h

# -----------
numerical_grad = [dfda, dfdb, dfdc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


In [18]:
# there is an alternative formula that provides a much better numerical
# approximation to the derivative of a function.
# learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative
# implement it. confirm that for the same step size h this version gives a
# better approximation.

# -----------
h = 0.001
dfda = (f(2 + h, 3, 4) - f(2 - h, 3, 4)) / (2*h)
dfdb = (f(2, 3 + h, 4) - f(2, 3 - h, 4)) / (2*h)
dfdc = (f(2, 3, 4 + h) - f(2, 3, 4 - h)) / (2*h)

# -----------
numerical_grad2 = [dfda, dfdb, dfdc] # TODO
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")


OK for dim 0: expected -12.353553390593273, yours returns -12.353554401639766
OK for dim 1: expected 10.25699027111255, yours returns 10.256994551617105
OK for dim 2: expected 0.0625, yours returns 0.06250000390650712


## section 2: support for softmax

In [75]:
# Value class starter code, with many functions taken out
from math import exp, log

class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out

  # ------
  # re-implement all the other functions needed for the exercises below
  # your code here
  # TODO
  # ------

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out._backward = _backward
    return out

  def exp(self):
    out = Value(math.e**self.data, (self,), 'exp')

    def _backward():
      self.grad += math.e**self.data * out.grad

    out._backward=_backward
    return out

  def log(self):
    out = Value(math.log(self.data), (self,), 'log')

    def _backward():
      self.grad += 1/self.data * out.grad

    out._backward = _backward
    return out

  def __radd__(self, other):
    return self + other

  def __truediv__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data / other.data, (self, other), '/')

    def _backward():
      self.grad += 1/other.data * out.grad # d/dx(x/y)=1/y
      other.grad += -self.data * other.data**-2 * out.grad # d/dx(y/x)=-y*x**-2

    out._backward = _backward
    return  out

  def __neg__(self):
    return self * -1

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [76]:
# without referencing our code/video __too__ much, make this cell work
# you'll have to implement (in some cases re-implemented) a number of functions
# of the Value object, similar to what we've seen in the video.
# instead of the squared error loss this implements the negative log likelihood
# loss, which is very often used in classification.

# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  print(counts)
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")


[Value(data=1.0), Value(data=20.085536923187664), Value(data=0.1353352832366127), Value(data=2.718281828459045)]
2.1755153626167143
OK for dim 0: expected 0.041772570515350445, yours returns 0.04177257051535045
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.00565330266221633
OK for dim 3: expected -0.8864503806400986, yours returns -0.886450380640099


In [18]:
# verify the gradient using the torch library
# torch should give you the exact same gradient
import torch

t_logits = torch.tensor([0.0, 3.0, -2.0, 1.0]); t_logits.requires_grad = True
t_probs = torch.nn.functional.softmax(t_logits, dim=0)
t_loss = -torch.log(t_probs[3])

t_loss.backward()
print(t_loss.item())

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(t_logits.grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {t_logits.grad[dim]}")

2.1755154132843018
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772566735744476
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390244841575623
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302650898695
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864504098892212
